In [ ]:
from operator import itemgetter
from dotenv import load_dotenv

from langchain.memory import ConversationBufferMemory
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
)
from langchain_community.callbacks.manager import get_openai_callback
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai.chat_models.azure import AzureChatOpenAI
from langchain_together.llms import Together

load_dotenv()

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
model = ChatOpenAI(model="gpt-4-turbo-preview", temperature=0.7, max_tokens=4096)
# model = AzureChatOpenAI(model="gpt-4-turbo-preview", temperature=0.7, max_tokens=4096)
# model = Together(model="mistralai/Mixtral-8x7B-Instruct-v0.1", temperature=0.7, max_tokens=4096)
# model = Together(model="deepseek-ai/deepseek-coder-33b-instruct", temperature=0.7, max_tokens=4096)

system_prompt = """
"""

prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{question}"),
    ]
)

chain = (
    RunnablePassthrough.assign(chat_history=RunnableLambda(memory.load_memory_variables) | itemgetter("chat_history"))
    | prompt
    | model
    | StrOutputParser()
)


def get_answer(question):
    with get_openai_callback() as callback:
        response = chain.invoke({"question": question})
        print(callback, end="\n\n")
    print(response)
    memory.save_context({"input": question}, {"output": response})

In [ ]:
get_answer("")